In [2]:
import pandas as pd
import numpy as np
import warnings
import pymssql
import locale
import glob
import os

warnings.filterwarnings('ignore')
pd.set_option('use_inf_as_na', True)
pd.options.display.max_columns = None
pd.set_option('float_format', '{:.2f}'.format)
locale.setlocale(locale.LC_ALL, 'ru_RU.UTF-8')

'ru_RU.UTF-8'

In [3]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [Код склада], [Название склада], [Артикул поставщика], [Внутренний артикул], [ID Сайта], [Название товара], [Остаток], [Цена Закупки за 1 шт. (упр сс с НДС за единицу)], [ШК], [Код товара]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'БК', N'ЦБ', N'СИ', N'ИФ', N'ПДК', N'ДК', N'ВЫРИЦА', N'ЧАСЫ')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [4]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Архив_отчеты\Продажи\По дням\\'
list_of_files = glob.glob(path + '*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

sklad = pd.read_excel(latest_file, sheet_name = 'Продажи по ассортименту', header = 2)

stock['Код склада'] = stock['Код склада'].astype('float64') 

sklad = sklad[['Unnamed: 0','Торговая марка', 'Юр лицо', 'Код склада', 'Дата открытия объекта', 'Код объекта', 'Город', 'Наименование объекта']]
sklad1 = sklad[['Код склада', 'Код объекта']].drop_duplicates().reset_index(drop=True)
sklad1 = sklad1[sklad1['Код объекта'].notna()]

sklad1['Код объекта'] = sklad1['Код объекта'].astype(str)
sklad1['8'] = sklad1['Код объекта'].apply(lambda x: x[0])
sklad1 = sklad1[sklad1['8'] != '8']

stock = stock.merge(sklad1, left_on='Код склада', right_on = 'Код склада', how='inner')

stock = stock.drop(columns=['8'])

In [5]:
stock = stock[stock['ID Сайта'] != '']

In [6]:
stock['Код склада'] = stock['Код склада'].astype(int)

In [7]:
os.chdir(r'C:\\Users\Trenkin.Sergey\Desktop\sales')
extension = 'xlsx'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames

['2024-06-01_2024-06-09.xlsx',
 '2024-06-10_2024-06-19.xlsx',
 '2024-06-20_2024-06-30.xlsx']

In [8]:
sale = pd.DataFrame()
for name in all_filenames:
    sales = pd.read_excel(name)
    sale = pd.concat([sales, sale], ignore_index= True)

In [9]:
sale

,КБК,Дата,Продавец,Номер чека,ID продажи,Тов,Артикул поставщика,Количество,Номенклатура,Номер карты,Товарная группа,Товарное направление,Тип 1,Списание СберСпасибо,Группа наценки,Вес,Проба,Размер,ШК,Цена без скидки,Базовая скидка,Скидка в руб.,Название,Цена после скидки,Phone,Себестоимость,Сумма,Услуга,Общая сумма
0,3-09-049,2024-06-20 00:04:00,Мазур Ирина Александровна,4303.00,Ch2-000908816,ТОВ1172935,CR_BOX,1,Футляр,9000146727565,NaN,ФУТЛЯРЫ,ФУТЛЯР,0.00,NaN,0.00,NaN,0.00,NaN,10.00,0.00,0.00,КОРОБОЧКА 585*Золото,10.00,+79244399091,NaN,10.00,0.00,10.00
1,3-09-019,2024-06-20 00:04:00,Лебедь Виктория Николаевна,4009.00,Ch2-000908805,ТОВ1321918,12-72-0001-15442,1,Товар,9000214731111,СИ СЕРЬГИ РАЗН,СИ,СЕРЬГИ,0.00,СИ_СЕРЬГИ_СРЕДН,1.84,925.00,0.00,2078610587614.00,2149.00,40.00,755.30,СЕРЬГИ-ПУССЕТЫ С ФИА,755.30,(996)024-97-36,369.19,755.30,154.70,910.00
2,3-09-049,2024-06-20 00:04:00,Мазур Ирина Александровна,4303.00,Ch2-000908816,ТОВ0832988,КО 05-00,1,Товар,9000146727565,БК КОЛЬЦА ОБРУЧ,БК,КОЛЬЦО ОБРУЧАЛЬНОЕ,0.00,ИФ.БК_СР,3.26,585.00,21.00,2078607036937.00,64020.00,24.00,0.00,КОЛЬЦО ОБРУЧАЛЬНОЕ,24153.00,+79244399091,10312.73,24153.00,4947.00,29100.00
3,3-09-049,2024-06-20 00:04:00,Вербицкая Лилия Игоревна,663.00,Ch2-000908804,ТОВ1226219,35-0285-30-00#375,1,Товар,7670001198612,БК ПОДВЕС КУЛЬТ,БК,ПОДВЕС КУЛЬТОВЫЙ,0.00,ИФ_375,3.28,375.00,0.00,2078611003639.00,58157.00,45.00,0.00,КРЕСТ,14898.50,+79644462237,7127.55,14898.50,3051.50,17950.00
4,3-09-019,2024-06-20 00:04:00,Лебедь Виктория Николаевна,4009.00,Ch2-000908805,ТОВ1163703,ANTIBACTERIAL_WIPE,1,Аксессуар футляр,9000214731111,NaN,ДФРС,ЧС,0.00,NaN,0.00,NaN,0.00,NaN,10.00,0.00,10.00,САЛФЕТКА АНТИБАКТЕРИ,10.00,(996)024-97-36,NaN,10.00,0.00,10.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874266,6-01-132,2024-06-09 19:07:00,Крючкова Екатерина Николаевна,3618.00,Ch4-000548409,ТОВ1296927,2701588-00245,1,Товар,9000156425857,СИ АКСЕССУАР,СИ,БРОШЬ,297.47,СИ_АКС_СРЕДН,1.10,925.00,0.00,2078609515464.00,1650.00,40.00,464.80,БРОШЬ РОД.,464.80,+79515190038,253.06,464.80,95.20,560.00
874267,6-01-132,2024-06-09 19:07:00,Крючкова Екатерина Николаевна,3618.00,Ch4-000548409,ТОВ1261812,МТБ1,1,Футляр,9000156425857,ФУТЛЯРЫ,ФУТЛЯРЫ,ПРОЧЕЕ,41.60,NaN,0.00,NaN,0.00,NaN,65.00,0.00,65.00,ФУТЛЯР МЕШОЧЕК БРЕНД,65.00,+79515190038,NaN,65.00,0.00,65.00
874268,3-09-051,2024-06-09 23:46:00,Григорян Агнеса Арсеновна,1183.00,Ch2-000871636,ТОВ1163703,ANTIBACTERIAL_WIPE,1,Аксессуар футляр,7651045384070,NaN,ДФРС,ЧС,0.00,NaN,0.00,NaN,0.00,NaN,10.00,0.00,10.00,САЛФЕТКА АНТИБАКТЕРИ,10.00,+79147809472,NaN,10.00,0.00,10.00
874269,3-09-051,2024-06-09 23:46:00,Григорян Агнеса Арсеновна,1183.00,Ch2-000871636,ТОВ1315471,Ср925Р-5А3100000Ф1,1,Товар,7651045384070,СИ СЕРЬГИ ИФ,СИ,СЕРЬГИ,0.00,СИ_СЕРЬГИ_СРЕДН,2.35,925.00,0.00,2078610244729.00,3315.00,40.00,929.60,СЕРЬГИ С ФИАН. РОД.,929.60,+79147809472,472.50,929.60,190.40,1120.00


In [10]:
sale.columns = ['КБК', 'Дата', 'Продавец', 'Номер чека', 'ID продажи', 'Тов',
    'article', 'Количество', 'Номенклатура', 'Номер карты',
    'Товарная группа', 'Товарное направление', 'Тип 1',
    'Списание СберСпасибо', 'Группа наценки', 'Вес', 'Проба', 'Размер',
    'ШК', 'Цена без скидки', 'Базовая скидка', 'Скидка в руб.', 'Название',
    'Цена после скидки', 'Phone', 'Себестоимость', 'Сумма', 'Услуга',
    'Общая сумма']

In [11]:
articles = stock['Артикул поставщика'].tolist()

In [12]:
sale = sale.query('article in @articles')

In [13]:
sale = sale.groupby(by = 'article', as_index=False)[['Количество', 'Общая сумма']].sum()

In [14]:
sale.columns = ['Артикул поставщика', 'Количество продаж', 'Общая сумма продаж за июнь']

In [15]:
center = stock[stock['Код склада'] == 3075]
center = center.groupby(by = 'ID Сайта', as_index=False)['Остаток'].sum()
center.columns = ['ID Сайта', 'Остаток на центральном складе']

In [16]:
kolambus = stock[stock['Код склада'] == 5442]
kolambus = kolambus.groupby(by = 'ID Сайта', as_index=False)['Остаток'].sum()
kolambus.columns = ['ID Сайта', 'Остаток на складе коламбуса']

In [17]:
federal = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Федеральные склады 11.06.xlsx')

In [18]:
stock.rename(columns={'Название склада': 'name'}, inplace=True)

In [19]:
federal = federal['outlet_name'].tolist() 

In [20]:
federal = stock.query('name in @federal')

In [21]:
federal = federal.groupby(by = 'ID Сайта', as_index=False)['Остаток'].sum()
federal.columns = ['ID Сайта', 'Остаток на федеральных складах']

In [22]:
stock = stock.merge(federal, how='outer', on='ID Сайта')
stock = stock.merge(kolambus, how='outer', on='ID Сайта')
stock = stock.merge(center, how='outer', on='ID Сайта')

In [23]:
conn = pymssql.connect(server='spb-s-sql-dwh', database='DWH_DEV', charset='WINDOWS-1251')
cursor = conn.cursor()

sql = f'''
SELECT [article], [sort]
from [DWH_DEV].[ecom].[catalog_product]
'''

sql_query = pd.read_sql_query(sql, conn)
sort = pd.DataFrame(sql_query)

In [24]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5766648 entries, 0 to 5766647
Data columns (total 14 columns):
 #   Column                                           Dtype  
---  ------                                           -----  
 0   Код склада                                       int32  
 1   name                                             object 
 2   Артикул поставщика                               object 
 3   Внутренний артикул                               object 
 4   ID Сайта                                         object 
 5   Название товара                                  object 
 6   Остаток                                          float64
 7   Цена Закупки за 1 шт. (упр сс с НДС за единицу)  float64
 8   ШК                                               object 
 9   Код товара                                       object 
 10  Код объекта                                      object 
 11  Остаток на федеральных складах                   float64
 12  Остаток на скл

In [25]:
sort.rename(columns={'article': 'ID Сайта'}, inplace=True)

In [26]:
stock = stock.merge(sort, how='outer', on='ID Сайта')

In [27]:
sale

,Артикул поставщика,Количество продаж,Общая сумма продаж за июнь
0,0-017,61,43350.00
1,00-01-2336,1,6390.00
2,00-02-0604,1,9315.00
3,00-03-0858,4,25200.00
4,00002Б,3,1050.00
...,...,...,...
38852,с9-16,64,64542.00
38853,ск-25,7,40560.00
38854,ск-30,8,31750.00
38855,ск-40,3,19670.00


In [28]:
stock = stock.merge(sale, how='outer', on='Артикул поставщика')

In [29]:
summ = stock.groupby(by='ID Сайта', as_index=False)['Остаток'].sum()

In [30]:
summ.rename(columns={'Остаток': 'Общий остаток на всех складах'}, inplace=True)

In [31]:
stock = stock.merge(summ, how='left', on='ID Сайта')
stock = stock.drop_duplicates(subset=['ID Сайта', 'Общий остаток на всех складах'])

In [32]:
stock['Остаток на складе коламбуса'].fillna(0, inplace=True)
stock['Остаток на федеральных складах'].fillna(0, inplace=True)
stock['Остаток на центральном складе'].fillna(0, inplace=True)

In [33]:
stock['Сумма в себестоимости на остатках ОБЩ'] = stock['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'] * stock['Общий остаток на всех складах']
stock['Сумма в себестоимости на остатках ФЕД+КОЛАМБУС+ЦЕНТР'] = (stock['Остаток на складе коламбуса'] + stock['Остаток на федеральных складах'] + stock['Остаток на центральном складе']) * stock['Цена Закупки за 1 шт. (упр сс с НДС за единицу)']

In [34]:
stock

,Код склада,name,Артикул поставщика,Внутренний артикул,ID Сайта,Название товара,Остаток,Цена Закупки за 1 шт. (упр сс с НДС за единицу),ШК,Код товара,Код объекта,Остаток на федеральных складах,Остаток на складе коламбуса,Остаток на центральном складе,sort,Количество продаж,Общая сумма продаж за июнь,Общий остаток на всех складах,Сумма в себестоимости на остатках ОБЩ,Сумма в себестоимости на остатках ФЕД+КОЛАМБУС+ЦЕНТР
0,5780.00,"Воронеж, Ленинский, 174 п, ""ТРЦ МАКСИМИР""",2-162-3,2-162-3_СИ,3987031,Крест с чернением,1.00,320.63,2078609817465,ТОВ0709748,2-04-005,0.00,0.00,0.00,0,70.00,54220.00,371.00,118953.73,0.00
371,5780.00,"Воронеж, Ленинский, 174 п, ""ТРЦ МАКСИМИР""",1-004-3,1-004-3_СИ,4783178,Крест с чернением,1.00,285.84,2078611166624,ТОВ0709788,2-04-005,1.00,1.00,0.00,0,49.00,28070.00,276.00,78891.84,571.68
647,5780.00,"Воронеж, Ленинский, 174 п, ""ТРЦ МАКСИМИР""",БР-020Ч,БР-020Ч_СИ,3870759,Браслет Бисмарк черн.,1.00,1462.09,2078611276873,ТОВ0711905,2-04-005,13.00,6.00,0.00,21935,152.00,301132.00,1681.00,2457773.29,27779.71
2328,5780.00,"Воронеж, Ленинский, 174 п, ""ТРЦ МАКСИМИР""",3-135Ч,3-135Ч_СИ,2075933,Крест с чернением,1.00,2517.59,2078607855308,ТОВ0712090,2-04-005,3.00,1.00,0.00,11690,139.00,676820.00,649.00,1633915.91,10070.36
2977,5780.00,"Воронеж, Ленинский, 174 п, ""ТРЦ МАКСИМИР""",3-276Ч,3-276Ч_СИ,3964374,Крест с чернением,1.00,310.20,2078610792468,ТОВ0712120,2-04-005,5.00,1.00,0.00,0,128.00,66830.00,730.00,226446.00,1861.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5868820,NaN,NaN,NaN,NaN,2235328,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0,NaN,NaN,0.00,NaN,NaN
5868821,NaN,NaN,NaN,NaN,7741803,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0,NaN,NaN,0.00,NaN,NaN
5868822,NaN,NaN,NaN,NaN,2212903,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0,NaN,NaN,0.00,NaN,NaN
5868823,NaN,NaN,NaN,NaN,1357200,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0,NaN,NaN,0.00,NaN,NaN


In [35]:
images = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Артикул-фото 2024.06.11_.xlsx')

In [36]:
images = images[['article', 'Ссылка на изображение', 'published']]

In [37]:
images.rename(columns={'article': 'ID Сайта'}, inplace=True)

In [38]:
images['ID Сайта'] = images['ID Сайта'].astype(str)

In [39]:
stock = stock.merge(images, how='outer', on='ID Сайта')

In [40]:
stock = stock.drop(columns=['Остаток', 'Код склада'])

In [41]:
ecom = pd.read_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\ecom_stock_raw\Остатки 2024.07.05.xlsx')

In [42]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\discounts_zolotoy\\'
list_of_files = glob.glob(path + '\*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

discount_ecom = pd.read_excel(latest_file)

In [43]:
discount_ecom.columns = ['article', 'Опубликован', 'discount_value_promotions_1','discount_value_promotions_2', 'discount_value_promotions_3','discount_value_promotions_4']
ecom = ecom.merge(discount_ecom, how='left', on='article')

In [44]:
def discount_index(discount):
    try:
        return 1 - discount / 100
    except:
        return 1
    
ecom.discount_value_promotions_1 = ecom.discount_value_promotions_1.apply(discount_index)

ecom['Цена со скидками Ecom'] = np.where(
    (ecom['price'] == ecom['discount_price']) & (ecom['discount_value_promotions_1'] == 1), ecom['discount_price'] * ecom['discount_value_promotions_1'], 
    np.where(
        ecom['discount_value_promotions_1'] == 0.5, ecom['discount_price'] * ecom['discount_value_promotions_1'], ecom['discount_price'] * ecom['discount_value_promotions_1'] * 0.8
    )
)

In [45]:
ecom['article'] = ecom['article'].astype(str)
ecom = ecom[['article', 'Цена со скидками Ecom']]

In [46]:
ecom.columns = ['ID Сайта', 'Цена со скидками Ecom']
stock = stock.merge(ecom, how='left', on='ID Сайта')

In [47]:
stock = stock.drop_duplicates(subset=['ID Сайта'])

In [48]:
stock = stock.drop(columns=['name', 'Код объекта', 'Цена Закупки за 1 шт. (упр сс с НДС за единицу)'])

In [49]:
stock.columns =['Артикул поставщика', 'Внутренний артикул', 'ID Сайта',
    'Название товара', 'ШК', 'Код товара', 'Остаток на федеральных складах',
    'Остаток на складе коламбуса', 'Остаток на центральном складе', 'Ранг на сайте',
    'Количество продаж', 'Общая сумма продаж за июнь',
    'Общий остаток на всех складах',
    'Сумма в себестоимости на остатках ОБЩ',
    'Сумма в себестоимости на остатках ФЕД+КОЛАМБУС+ЦЕНТР',
    'Ссылка на изображение', 'Опубликован', 'Цена со скидками Ecom']

In [50]:
zapret = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\запрет Артикулы к размещению.xlsx')['Артикул сайта'].astype(str).tolist()

In [51]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
Index: 196830 entries, 0 to 401617
Data columns (total 18 columns):
 #   Column                                                Non-Null Count   Dtype  
---  ------                                                --------------   -----  
 0   Артикул поставщика                                    94653 non-null   object 
 1   Внутренний артикул                                    94653 non-null   object 
 2   ID Сайта                                              196830 non-null  object 
 3   Название товара                                       94653 non-null   object 
 4   ШК                                                    94653 non-null   object 
 5   Код товара                                            94653 non-null   object 
 6   Остаток на федеральных складах                        196830 non-null  float64
 7   Остаток на складе коламбуса                           196830 non-null  float64
 8   Остаток на центральном складе                    

In [52]:
stock['Запрет к размещению'] = np.nan
for i in stock.index:
    if stock['ID Сайта'][i] in zapret:
        stock['Запрет к размещению'][i] = True
    else:
        stock['Запрет к размещению'][i] = False

In [53]:
zapret

['1931032',
 '1191901',
 '1116455',
 '1473647',
 '1864080',
 '1046937',
 '1518445',
 '1524395',
 '1389377',
 '1045798',
 '2224191',
 '1858904',
 '1463937',
 '1952633',
 '1035638',
 '1509411',
 '2230937',
 '1726581',
 '1750930',
 '1742870',
 '1203860',
 '1022701',
 '1915786',
 '1777828',
 '3550980',
 '1433494',
 '1173415',
 '8328749',
 '4765813',
 '8175129',
 '1383732',
 '1759484',
 '1875158',
 '8177955',
 '7981402',
 '8067677',
 '8177947',
 '1371490',
 '8151251',
 '8352772',
 '2244200',
 '1215546',
 '8177959',
 '1983721']

In [54]:
stock['Запрет к размещению'].value_counts()

Запрет к размещению
False    196786
True         44
Name: count, dtype: int64

In [55]:
stock.dropna(subset=['ШК'], inplace=True)

In [56]:
stock['Опубликован'].value_counts()

Опубликован
True     71577
False     4449
Name: count, dtype: int64

In [57]:
stock1 = stock[~stock['Ссылка на изображение'].astype(str).str.contains('http')]
# df = df[df['Ссылка на изображение'].astype(str).str.contains('http')]

In [67]:
stock2 = stock[(stock['Ссылка на изображение'].astype(str).str.contains('http')) & (stock['Запрет к размещению'] == False) & (stock['Опубликован'] == False)]

In [69]:
stock = pd.concat([stock1, stock2], ignore_index=True)

In [71]:
stock.to_excel('08.07.24 для контента.xlsx')

In [58]:
df[df['Опубликован'] < 1].to_excel(r'C:\Users\Trenkin.Sergey\Desktop\notebooks\\для контента неопубликованные.xlsx', index=False)

NameError: name 'df' is not defined